#### Sutton and Barto, Reinforcement Learning 2nd. Edition, page 92.

![Sutton and Barto, Reinforcement Learning 2nd. Edition.](./Figures/FirstVisitMCPrediction.png)
First-visit MC prediction, for estimating V

### Imports and Setup

This cell imports necessary libraries (`numpy` for numerical operations and `create_standard_grid` from a custom module `rlgridworld`) and sets a random seed for reproducibility.

In [21]:
from rlgridworld.standard_grid import create_standard_grid
import numpy as np

np.random.seed(42)

### `play_game` Function

This function simulates playing one episode of the grid world game according to a given policy. 

- It starts from a specified state (defaulting to (0,0)).
- In a loop, it determines the action based on the policy for the current state.
- It gets the reward for taking that action in the current state.
- It determines the next state using the `move` function.
- It records the (state, reward) pair.
- The loop continues until a terminal state is reached.
- Finally, it returns a list of (state, reward) tuples encountered during the episode.

In [22]:
def play_game(gw, policy, state=(0,0)):
    # default game starting point is state = (0,0) 
    # list of tuples that are (state, reward) pairs
    states_and_rewards = [] 
    converged = False
    while not converged:
        # get action from policy
        action = policy[state] 
        # find reward for the action
        reward = gw.get_reward_for_action(state, action)
        # move to the new state
        stateprime = move(state,action)
        # add new state and reward to the list
        states_and_rewards.append((state,reward))
        # if you have moved to a terminal state, then stop
        if gw.is_terminal(stateprime):
            converged = True
        # update state to new state
        state = stateprime
    return states_and_rewards

### `move` Function

This helper function calculates the next state based on the current state (i, j coordinates) and the chosen action ('up', 'down', 'left', 'right'). It assumes the action provided is valid for the state (i.e., doesn't lead into a wall immediately, although the `play_game` function handles terminal states).

In [23]:
def move(state, action): # only valid actions at states are sent to move
    i,j = state
    if action == 'left':
        j = j-1
    if action == 'right':
        j = j+1
    if action == 'down':
        i = i-1
    if action == 'up':
        i = i+1
    return (i,j)

### `compute_G` Function

This function calculates the discounted return Gt for each state visited during an episode.

- It takes the list of (state, reward) pairs from `play_game` and a discount factor `gamma`.
- It iterates through the episode's steps *in reverse*.
- For each step (s, r), it calculates the return G using the formula: G = r + gamma * G (where G is the return calculated from the subsequent steps).
- It stores the (state, G) pair.
- Finally, it reverses the list back to the original time order and returns the list of (state, return) pairs.

In [24]:
def compute_G(states_and_rewards, gamma):
    # function computes G(s) for states visited on trajectory of visited states during play_game()
    # input is states_and_rewards list returned from play_game
    # Note similarity in names: states_and_rewards is generated by play_game(). This function 
    # generates states_and_returns, which is a list of states and the associated G(s) value for the
    # state. 
    G = 0
    states_and_returns = [] # list of tuples that are (state, return) pairs
    for s, r in reversed(states_and_rewards):
        G = r + gamma*G
        states_and_returns.append((s,G))
    states_and_returns.reverse()
    return(states_and_returns)

### Create Grid World Environment

This cell initializes the standard grid world environment using the imported `create_standard_grid` function. The `gw` object represents the environment, holding information about states, actions, rewards, and transitions.

In [25]:
gw = create_standard_grid()

### Define Policy

This cell defines the policy (π) that we want to evaluate. It's represented as a dictionary where keys are state tuples (row, col) and values are the actions ('up', 'down', 'left', 'right', or '' for terminal/barrier states) to take in those states.

In [26]:
policy = { 
    (0,0):'up', (0,1):'right',(0,2):'right',(0,3):'up',
    (1,0):'up', (1,1):'', (1,2):'right', (1,3):'',
    (2,0):'right', (2,1):'right', (2,2):'right', (2,3):''
    }

### Initialization for Value Estimation

This cell sets up variables needed for the First-Visit Monte Carlo prediction algorithm:
- `gamma`: The discount factor (0.9) used in calculating returns.
- `all_states`: A list explicitly defining all possible states in the grid (excluding terminal/barrier states implicitly handled later).

In [27]:
gamma = 0.9
all_states = [
            (0,0), (0,1) ,(0,2), (0,3),
            (1,0), (1,1), (1,2), (1,3),
            (2,0), (2,1), (2,2), (2,3)
]

## Section 1: Single Episode Evaluation

This section demonstrates the core steps of First-Visit MC prediction using a single episode generated by following the defined policy from the default start state.

### Play One Episode

Calls the `play_game` function with the defined grid world (`gw`) and policy (`policy`) to simulate one complete episode, starting from the default state (0,0). The resulting sequence of (state, reward) pairs is stored in `states_and_rewards`.

In [28]:
states_and_rewards = play_game(gw, policy)

### Examine Episode Trajectory

Displays the `states_and_rewards` list generated in the previous step, showing the sequence of states visited and the immediate rewards received at each step during the single episode.

In [29]:
states_and_rewards

[((0, 0), 0.0), ((1, 0), 0.0), ((2, 0), 0.0), ((2, 1), 0.0), ((2, 2), 1.0)]

### Compute Returns (G) for the Episode

Calls the `compute_G` function to calculate the discounted return Gt for each state visited in the episode, using the `states_and_rewards` list and the discount factor `gamma`. The result is stored in `states_and_returns`.

In [30]:
states_and_returns = compute_G(states_and_rewards, gamma)

### Examine State Returns

Displays the `states_and_returns` list, showing each state visited during the episode paired with its calculated discounted return Gt.

In [31]:
states_and_returns

[((0, 0), 0.6561000000000001),
 ((1, 0), 0.7290000000000001),
 ((2, 0), 0.81),
 ((2, 1), 0.9),
 ((2, 2), 1.0)]

### Update State Values (First-Visit MC)

This cell implements the core logic of First-Visit Monte Carlo prediction for the single episode:
1. Initializes a dictionary `returns` to store the G values encountered for each state across episodes (though here, only one episode is processed).
2. Initializes an empty set `seen_states` to track states already visited *within this episode*.
3. Iterates through the `states_and_returns` list.
4. For each (state, G) pair, if the state has *not* been seen yet in this episode:
   - Appends the calculated return `G` to the list associated with that state in the `returns` dictionary.
   - Updates the value V(s) for that state in the grid world (`gw`) by taking the mean of all returns collected for that state so far (in this case, just the single G value).
   - Adds the state to the `seen_states` set to ensure only the *first* visit's return is used for this episode.

In [32]:
returns = {} # add code to not include terminal and barrier states to returns dictionary
for s in all_states:
    returns[s] = []
    
seen_states = set()
for s, G in states_and_returns:
    # print(s,G) # Uncomment to see step-by-step processing
    if s not in seen_states:
        returns[s].append(G)
        gw.set_value(s, np.mean(returns[s])) # mean is for stochastic (or averaging over episodes)
        seen_states.add(s)

### Show States Seen in Episode

Displays the `seen_states` set, showing which unique states were visited at least once during the single episode.

In [33]:
seen_states

{(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)}

### Display Updated Grid Values

Calls the `print_values` method on the grid world object (`gw`) to display the current estimated state values (V(s)). After processing only one episode, these values are based solely on the returns from that single trajectory.

In [34]:
gw.print_values()

-------------------------------------
|   0.81 |   0.90 |   1.00 |   0.00 |
-------------------------------------
|   0.73 |   0.00 |   0.00 |   0.00 |
-------------------------------------
|   0.66 |   0.00 |   0.00 |   0.00 |
-------------------------------------


### Show Collected Returns

Displays the `returns` dictionary, showing the list of returns collected for each state. After one episode, each visited state will have a list containing just one return value (the Gt from its first visit in that episode).

In [35]:
returns

{(0, 0): [0.6561000000000001],
 (0, 1): [],
 (0, 2): [],
 (0, 3): [],
 (1, 0): [0.7290000000000001],
 (1, 1): [],
 (1, 2): [],
 (1, 3): [],
 (2, 0): [0.81],
 (2, 1): [0.9],
 (2, 2): [1.0],
 (2, 3): []}

## Section 2: Multiple Episodes (Fixed Start)

This section repeats the process from Section 1 multiple times (10 episodes) but always starts from the same initial state (0,0). This highlights a potential issue with MC methods if exploration is insufficient.

### Run 10 Episodes (Fixed Start)

This cell runs the First-Visit MC prediction process for `num_episodes` (10) times:
1. **Loop:** Iterates 10 times.
2. **Play Episode:** Calls `play_game` starting from the default state (0,0).
3. **Compute Returns:** Calculates `states_and_returns` for the episode using `compute_G`.
4. **Initialize `returns`:** Resets the `returns` dictionary for *each episode* (Note: This is incorrect for accumulating averages across episodes. The `returns` dictionary should persist and accumulate outside the loop for proper MC averaging. The current code effectively only uses the *last* episode's returns to set the values).
5. **First-Visit Update:** Iterates through the episode's `states_and_returns`, updating `returns` and `gw` values only for the first visit to each state *within that specific episode*.

**Important Note:** Because `returns` is re-initialized inside the loop and `gw.set_value` overwrites the value based only on the current episode's first-visit return, this code doesn't correctly average returns across episodes as intended by the standard First-Visit MC algorithm. It essentially re-evaluates based on each new episode independently, overwriting previous results.

In [36]:
num_episodes = 10

for t in range(num_episodes):
    # Play episode from default start
    states_and_rewards = play_game(gw, policy)
    
    # Compute returns for the episode
    G = 0
    states_and_returns = []
    for s, r in reversed(states_and_rewards):
        G = r + gamma*G
        states_and_returns.append((s,G))
    states_and_returns.reverse()
    
    # *** Issue: returns should accumulate across episodes, not reset ***
    returns = {} 
    for s in all_states:
        returns[s] = []
    
    # First-visit update for the current episode
    seen_states = set()
    for s, G in states_and_returns:
        if s not in seen_states:
            returns[s].append(G) # Appends to the per-episode returns dict
            # Overwrites V(s) based only on this episode's first visit G
            gw.set_value(s, np.mean(returns[s])) 
            seen_states.add(s)

### Display Grid Values After 10 Fixed-Start Episodes

Prints the state values V(s) in the grid world after running the loop 10 times. Because the environment and policy are deterministic and the starting state is fixed, each episode follows the exact same path. Due to the implementation issue noted above (resetting `returns` and overwriting V(s) each time), the final values simply reflect the result from the 10th (last) identical episode.

In [37]:
gw.print_values()

-------------------------------------
|   0.81 |   0.90 |   1.00 |   0.00 |
-------------------------------------
|   0.73 |   0.00 |   0.00 |   0.00 |
-------------------------------------
|   0.66 |   0.00 |   0.00 |   0.00 |
-------------------------------------


### Explanation of Fixed-Start Results

This markdown cell correctly observes that since the grid world and policy are deterministic, starting from the same state always produces the same trajectory. Therefore, running 10 episodes yields the same result each time. It also notes that states in the bottom row are never visited because the policy immediately directs the agent upwards from the start state (0,0) and subsequent states visited.

## Section 3: Multiple Episodes (Random Starts)

This section addresses the exploration problem highlighted in Section 2 by using *exploring starts*. Each episode begins from a randomly chosen non-terminal, non-barrier state. This ensures that, over many episodes, all states have a chance of being visited.

### Run 100 Episodes (Random Starts)

This cell runs the First-Visit MC prediction process for `num_episodes` (100) times, incorporating exploring starts:
1. **Loop:** Iterates 100 times.
2. **Select Random Start:**
   - Picks a random index within the `all_states` list.
   - Gets the corresponding state.
   - **Validation:** Ensures the chosen state is not a barrier or terminal state; if it is, it re-picks until a valid start state is found.
3. **Play Episode:** Calls `play_game` starting from the randomly selected `state`.
4. **Compute Returns:** Calculates `states_and_returns` for the episode.
5. **Initialize `returns`:** Resets the `returns` dictionary for *each episode* (Same issue as Section 2: this prevents proper averaging across episodes).
6. **First-Visit Update:** Updates `returns` and `gw` values based only on the first visit to each state *within the current episode*.

**Note:** While exploring starts helps visit more states, the implementation flaw of resetting `returns` and overwriting V(s) each episode persists. The final V(s) will be heavily influenced by the *last* episode that happened to visit state s for the first time in that episode.

In [38]:
num_episodes = 100

for t in range(num_episodes):
    # select a random starting point in the list of all states
    random_indx = np.random.randint(0,len(all_states)) 
    # get the state 
    state = all_states[random_indx]
    # if the selected random state is a barrier state or a terminal state, then do it again
    while(gw.is_barrier(state) or gw.is_terminal(state)):
        random_indx = np.random.randint(0,len(all_states))
        state = all_states[random_indx]  
        
    # play the game from the selected state
    states_and_rewards = play_game(gw, policy, state=state)
    
    # compute states_and_returns
    G = 0
    states_and_returns = []
    for s, r in reversed(states_and_rewards):
        G = r + gamma*G
        states_and_returns.append((s,G))
    states_and_returns.reverse()
    
    # *** Issue: returns should accumulate across episodes, not reset ***
    returns = {} 
    for s in all_states:
        returns[s] = []
    
    # perform rest of calculation on the 
    seen_states = set()
    for s, G in states_and_returns:
        if s not in seen_states:
            returns[s].append(G)
            # Overwrites V(s) based only on this episode's first visit G
            gw.set_value(s, np.mean(returns[s])) 
            seen_states.add(s)

### Display Grid Values After 100 Random-Start Episodes

Prints the state values V(s) after 100 episodes with exploring starts. Because random starting points allow different trajectories to be explored, states that were previously unvisited (like those in the bottom row) now have estimated values. However, due to the implementation issue, these values are not the result of averaging returns across all 100 episodes but are influenced primarily by the last episode that visited each state.

In [39]:
gw.print_values()

-------------------------------------
|   0.81 |   0.90 |   1.00 |   0.00 |
-------------------------------------
|   0.73 |   0.00 |  -1.00 |   0.00 |
-------------------------------------
|   0.66 |  -0.81 |  -0.90 |  -1.00 |
-------------------------------------


### Display Policy

Calls the `print_policy` method on the grid world object (`gw`) to display the input policy that was being evaluated throughout the notebook. This helps visualize the actions taken in each state.

In [40]:
gw.print_policy(policy)

-------------------------------------
|  Right |  Right |  Right |        |
-------------------------------------
|     Up |        |  Right |        |
-------------------------------------
|     Up |  Right |  Right |     Up |
-------------------------------------


### Final Observation

This markdown cell notes that the values obtained using exploring starts appear more reasonable, particularly for the previously unvisited lower row states, compared to the fixed-start approach. It implies that the values now better reflect the expected returns under the given policy when starting from various states.